In [274]:
import pandas as pd
import os
import numpy as np

In [275]:
data1 = pd.read_csv('pulsar_stars.csv')
data1.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [276]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0    Mean of the integrated profile                17898 non-null  float64
 1    Standard deviation of the integrated profile  17898 non-null  float64
 2    Excess kurtosis of the integrated profile     17898 non-null  float64
 3    Skewness of the integrated profile            17898 non-null  float64
 4    Mean of the DM-SNR curve                      17898 non-null  float64
 5    Standard deviation of the DM-SNR curve        17898 non-null  float64
 6    Excess kurtosis of the DM-SNR curve           17898 non-null  float64
 7    Skewness of the DM-SNR curve                  17898 non-null  float64
 8   target_class                                   17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


In [277]:
np.all(np.isfinite(data1)) #Testing dataset

True

In [278]:
np.all(np.isnan(data1)) #Testing dataset

False

In [279]:
#data1 = data1 .dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 


In [280]:
data1["target_class"].value_counts()

0    16259
1     1639
Name: target_class, dtype: int64

In [281]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["target_class"]):
    strat_train_set = data1.loc[train_index]
    strat_test_set = data1.loc[test_index]

In [282]:
 strat_test_set["target_class"].value_counts()

0    4065
1     410
Name: target_class, dtype: int64

In [283]:
 strat_train_set["target_class"].value_counts()

0    12194
1     1229
Name: target_class, dtype: int64

In [284]:
train_set = strat_train_set.drop("target_class", axis=1) # drop labels for training set

train_labels = strat_train_set["target_class"].copy()
test_set = strat_test_set.drop("target_class", axis=1) # drop labels for testing set
test_labels = strat_test_set["target_class"].copy()

In [285]:
train_labels

15026    0
14795    0
5940     0
3002     0
12431    0
        ..
6349     0
12584    0
15650    0
4596     0
5560     1
Name: target_class, Length: 13423, dtype: int64

In [286]:
!pip install fuzzy-c-means
from fcmeans import FCM
tr_set=train_set.T
fcm = FCM(n_clusters=4,random_state=42)
fcm.fit(tr_set.to_numpy())
tr_fin  = fcm.centers.T


In [287]:
te_set=test_set.T
fcm = FCM(n_clusters=4, random_state=42)
fcm.fit(te_set.to_numpy())
te_fin  = fcm.centers.T

In [288]:
np.any(np.isnan(te_fin)) #Testing dataset


False

In [289]:
np.all(np.isfinite(te_fin)) #Testing dataset

True

In [290]:
#It replaces the infinity values with high finite values and the nan values with numbers

#tr_fin = np.nan_to_num(tr_fin)
#te_fin = np.nan_to_num(te_fin)





In [291]:
tr_fin.shape

(13423, 4)

In [292]:
train_labels.shape

(13423,)

In [293]:
from sklearn.metrics import matthews_corrcoef
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
parameters = {'min_samples_leaf':[0.1,1,2,3],'max_depth':[10,50,100], 'criterion':['gini', 'entropy'], 'random_state':[42]}
clf2 = DecisionTreeClassifier()
clf = GridSearchCV(clf2, parameters, scoring = 'balanced_accuracy', cv=10)

res1=clf.fit(tr_fin, train_labels)
res1.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [294]:
res1.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'random_state': 42}

In [295]:
res1.best_score_

0.8912559354163723

In [296]:
y_test_pred=res1.best_estimator_.predict(te_fin)

In [297]:
import sklearn
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100

97.07262569832402

In [298]:
#error rate=100-accuracy
100-(sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100)

2.9273743016759823

In [299]:
matthews_corrcoef(test_labels, y_test_pred)*100

81.74976903273021

In [300]:
from sklearn.metrics import precision_score
sklearn.metrics.precision_score(test_labels, y_test_pred, pos_label=1)*100

87.6010781671159

In [301]:
from sklearn.metrics import recall_score
sklearn.metrics.recall_score(test_labels, y_test_pred, pos_label=1)*100

79.26829268292683

In [302]:
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(test_labels, y_test_pred, pos_label=1)*100

83.22663252240716